# Stock NeurIPS2018 Part 3. Backtest
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*.

This is the third and last part of the NeurIPS2018 series, introducing how to use use the agents we trained to do backtest, and compare with baselines such as Mean Variance Optimization and DJIA index.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [ ]:
# ===========================
# Suppress Warnings
# ===========================
import warnings
warnings.filterwarnings("ignore")

# ===========================
# Standard Libraries
# ===========================
import os
import sys
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# matplotlib.use('Agg')  

# ===========================
# Enable Inline Plotting (Jupyter)
# ===========================
%matplotlib inline

# ===========================
# FinRL Imports
# ===========================
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split

# ===========================
# Custom Imports (model.py)
# ===========================
sys.path.append(os.path.abspath("."))  
from models import DRLEnsembleAgent

sys.path.append("../FinRL-Library")


# Part 2. Backtesting

In [175]:
def process_csv_to_features(csv_path):
    # Step 1: Load Data
    df = pd.read_csv(csv_path)

    # Step 2: Identify 5-day and 7-day tickers
    day_values_per_tic = df.groupby('tic')['day'].apply(lambda x: sorted(x.unique())).reset_index()
    day_values_per_tic.columns = ['tic', 'unique_days']

    tics_5day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(5)))]['tic']
    tics_7day = day_values_per_tic[day_values_per_tic['unique_days'].apply(lambda x: x == list(range(7)))]['tic']

    df_5day_full = df[df['tic'].isin(tics_5day)]
    df_7day_full = df[df['tic'].isin(tics_7day)]

    # Step 3: Apply Technical Indicators
    fe_ti = FeatureEngineer(
        use_technical_indicator=True,
        use_turbulence=False,
        user_defined_feature=False
    )
    df_5day_full = fe_ti.preprocess_data(df_5day_full)
    if not df_7day_full.empty:
        df_7day_full = fe_ti.preprocess_data(df_7day_full)
    else:
        print("[Info] df_7day_full is empty. Skipping technical indicators.")

    # Step 4: Combine and Clean Index
    combined_df = pd.concat([df_5day_full, df_7day_full], ignore_index=False)
    combined_df.index = range(len(combined_df))

    # Step 5: Remove dates with only one ticker
    combined_df['date'] = pd.to_datetime(combined_df['date'])
    combined_df = combined_df[combined_df.groupby('date')['date'].transform('count') > 1]
    combined_df = combined_df.sort_values(['date', 'tic']).reset_index(drop=True)

    # Step 6: Apply Turbulence Feature
    fe_turb = FeatureEngineer(
        use_technical_indicator=False,
        use_turbulence=True,
        user_defined_feature=False
    )
    processed = fe_turb.preprocess_data(combined_df)

    # Step 7: Final Cleaning
    processed = processed.copy()
    processed = processed.fillna(0)
    processed = processed.replace(np.inf, 0)

    return processed


In [176]:
processed_0 = process_csv_to_features('2007-2025_no_crypto.csv')
processed_1 = process_csv_to_features('2015-2025_crypto.csv')
processed_2 = process_csv_to_features('2015-2025_no_crypto.csv')

Successfully added technical indicators
[Info] df_7day_full is empty. Skipping technical indicators.
Successfully added turbulence index
Successfully added technical indicators
Successfully added technical indicators
Successfully added turbulence index
Successfully added technical indicators
[Info] df_7day_full is empty. Skipping technical indicators.
Successfully added turbulence index


# Part 3: Mean Variance Optimization

Mean Variance optimization is a very classic strategy in portfolio management. Here, we go through the whole process to do the mean variance optimization and add it as a baseline to compare.

First, process dataframe to the form for MVO weight calculation.

In [177]:
def process_df_for_mvo(df):
    df = df.sort_values(['date', 'tic'], ignore_index=True)[['date', 'tic', 'close']]
    all_tickers = sorted(df['tic'].unique())
    ticker_index = {tic: idx for idx, tic in enumerate(all_tickers)}
    stock_dimension = len(all_tickers)

    mvo = pd.DataFrame(columns=all_tickers)

    grouped = df.groupby('date')
    for date, group in grouped:
        row = [np.nan] * stock_dimension
        for _, row_data in group.iterrows():
            row[ticker_index[row_data['tic']]] = row_data['close']
        if not any(pd.isna(row)):  # only include dates with all tickers
            mvo.loc[date] = row

    return mvo


### Helper functions for mean returns and variance-covariance matrix

In [178]:
# Codes in this section partially refer to Dr G A Vijayalakshmi Pai

# https://www.kaggle.com/code/vijipai/lesson-5-mean-variance-optimization-of-portfolios/notebook

def StockReturnsComputing(StockPrice, Rows, Columns):
  import numpy as np
  StockReturn = np.zeros([Rows-1, Columns])
  for j in range(Columns):        # j: Assets
    for i in range(Rows-1):     # i: Daily Prices
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100

  return StockReturn

In [179]:
import os
import pandas as pd
import numpy as np

def run_naive_portfolio_pipeline(df, 
                                  start_date, 
                                  end_date, 
                                  initial_fund=1_000_000, 
                                  buy_cost_pct=0.0, 
                                  output_return_csv='df_daily_return_naive.csv',
                                  original_csv_path='data.csv',
                                  model_name='naive'):
    """
    Compute naive equal-weighted portfolio returns and save outputs in structured folders.

    Parameters:
    -----------
    df : DataFrame
        Processed dataframe containing 'date', 'tic', and 'close'.
    start_date : str
        Start date in 'YYYY-MM-DD' format.
    end_date : str
        End date in 'YYYY-MM-DD' format.
    initial_fund : float
        Starting capital.
    buy_cost_pct : float
        Transaction cost percentage on initial buy.
    output_return_csv : str
        Filename for saving daily returns CSV.
    original_csv_path : str
        The path of the original CSV to derive folder name.
    model_name : str
        Name of the model to create subfolder (e.g., 'naive').
    """

    # === Step 0: Setup Folder Structure ===
    base_name = os.path.splitext(os.path.basename(original_csv_path))[0]
    target_folder = os.path.join(base_name, model_name)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"[INFO] Created folder: {target_folder}")

    # Step 1: Filter trade data
    trade_df = data_split(df, start_date, end_date).reset_index(drop=True)
    trade_df = trade_df.sort_values(['date', 'tic']).reset_index(drop=True)

    # Step 2: Process for MVO-like structure
    trade_mvo = process_df_for_mvo(trade_df)

    if trade_mvo.empty or len(trade_mvo) < 1:
        raise ValueError("Insufficient data for naive portfolio.")

    tickers = trade_mvo.columns.tolist()
    stock_dimension = len(tickers)

    # Step 3: Equal weight allocation
    first_prices = trade_mvo.iloc[0].to_numpy()
    if np.any(first_prices == 0):
        raise ValueError("Zero price detected in first trading day.")

    equal_weight = 1.0 / stock_dimension
    allocation_per_asset = initial_fund * (1 - buy_cost_pct) * equal_weight
    shares = allocation_per_asset / first_prices

    # Step 4: Calculate portfolio value
    portfolio_values = trade_mvo @ shares
    result_df = pd.DataFrame({
        "date": trade_mvo.index,
        "account_value": portfolio_values
    })

    # Step 5: Compute daily returns
    result_df["date"] = pd.to_datetime(result_df["date"])
    result_df.set_index("date", inplace=True)

    df_daily_return = result_df.copy()
    df_daily_return["daily_return"] = df_daily_return["account_value"].pct_change()
    df_daily_return = df_daily_return.reset_index()

    df_daily_return.loc[0, "daily_return"] = 0.0
    df_daily_return = df_daily_return[["date", "daily_return"]]

    # Step 6: Export to /data/naive/
    csv_full_path = os.path.join(target_folder, output_return_csv)
    df_daily_return.to_csv(csv_full_path, index=False)
    print(f"[INFO] Naive portfolio daily returns saved to {csv_full_path}")

    return df_daily_return


In [180]:
TRADE_START_DATE = '2023-04-05'
TRADE_END_DATE = '2025-04-10'

df_daily_return_naive = run_naive_portfolio_pipeline(
    df=processed_0,
    start_date=TRADE_START_DATE,
    end_date=TRADE_END_DATE,
    initial_fund=1_000_000,
    buy_cost_pct=0.001,  
    output_return_csv='df_daily_return_naive.csv',
    original_csv_path='2007-2025_no_crypto.csv',   
    model_name='naive'             
)
df_daily_return_naive = run_naive_portfolio_pipeline(
    df=processed_1,
    start_date=TRADE_START_DATE,
    end_date=TRADE_END_DATE,
    initial_fund=1_000_000,
    buy_cost_pct=0.001,  
    output_return_csv='df_daily_return_naive.csv',
    original_csv_path='2015-2025_crypto.csv',   
    model_name='naive'             
)
df_daily_return_naive = run_naive_portfolio_pipeline(
    df=processed_2,
    start_date=TRADE_START_DATE,
    end_date=TRADE_END_DATE,
    initial_fund=1_000_000,
    buy_cost_pct=0.001,  
    output_return_csv='df_daily_return_naive.csv',
    original_csv_path='2015-2025_no_crypto.csv',   
    model_name='naive'             
)

# processed_0 = process_csv_to_features('2007-2025_no_crypto.csv')
# processed_1 = process_csv_to_features('2015-2025_crypto.csv')
# processed_2 = process_csv_to_features('2015-2025_no_crypto.csv')


[INFO] Created folder: 2007-2025_no_crypto/naive
[INFO] Naive portfolio daily returns saved to 2007-2025_no_crypto/naive/df_daily_return_naive.csv
[INFO] Created folder: 2015-2025_crypto/naive
[INFO] Naive portfolio daily returns saved to 2015-2025_crypto/naive/df_daily_return_naive.csv
[INFO] Created folder: 2015-2025_no_crypto/naive
[INFO] Naive portfolio daily returns saved to 2015-2025_no_crypto/naive/df_daily_return_naive.csv


### Calculate the weights for mean-variance

In [181]:
from pypfopt.efficient_frontier import EfficientFrontier



In [182]:
import os
import pandas as pd
import numpy as np
from pypfopt.efficient_frontier import EfficientFrontier

def run_mvo_portfolio_pipeline(df, 
                                train_start_date, 
                                train_end_date, 
                                trade_start_date, 
                                trade_end_date, 
                                initial_fund=1_000_000, 
                                weight_bounds=(0.01, 0.25), 
                                output_return_csv='df_daily_return_mvo.csv',
                                original_csv_path='data.csv'):
    """
    Compute MVO portfolio returns, save outputs in structured folders.

    Folder Structure: 
    - <csv_name>/<model_name>/ (e.g., data/mvo/)
    """

    # === Step 0: Setup Folder Structure ===
    base_name = os.path.splitext(os.path.basename(original_csv_path))[0]
    model_name = "mvo"
    target_folder = os.path.join(base_name, model_name)

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"[INFO] Created folder: {target_folder}")

    # Adjust output CSV path to be within the folder
    output_csv_full_path = os.path.join(target_folder, output_return_csv)

    # === Step 1: Split Data ===
    train = data_split(df, train_start_date, train_end_date).reset_index(drop=True)
    trade = data_split(df, trade_start_date, trade_end_date).reset_index(drop=True)

    # === Step 2: Process Data for MVO ===
    StockData = process_df_for_mvo(train)
    TradeData = process_df_for_mvo(trade)

    # === Step 3: Compute Returns, Mean, Covariance ===
    arStockPrices = np.asarray(StockData)
    Rows, Cols = arStockPrices.shape
    arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)

    meanReturns = np.mean(arReturns, axis=0)
    covReturns = np.cov(arReturns, rowvar=False)

    # === Step 4: Perform Mean-Variance Optimization ===
    ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=weight_bounds)
    ef_mean.max_sharpe()
    cleaned_weights_mean = ef_mean.clean_weights()

    # Optional: Save weights to file
    weights_path = os.path.join(target_folder, "optimized_weights.json")
    pd.Series(cleaned_weights_mean).to_json(weights_path)
    print(f"[INFO] Optimized weights saved to {weights_path}")

    # === Step 5: Allocate Capital ===
    allocation = np.array([initial_fund * cleaned_weights_mean[tic] for tic in cleaned_weights_mean.keys()])
    first_prices = TradeData.iloc[0].to_numpy()
    shares = allocation / first_prices

    # === Step 6: Compute Portfolio Value Over Time ===
    portfolio_values = TradeData @ shares
    MVO_result = pd.DataFrame({
        "date": TradeData.index,
        "account_value": portfolio_values
    })

    # === Step 7: Calculate Daily Returns ===
    MVO_result["date"] = pd.to_datetime(MVO_result["date"])
    MVO_result.set_index("date", inplace=True)

    df_daily_return = MVO_result.copy()
    df_daily_return["daily_return"] = df_daily_return["account_value"].pct_change()

    df_daily_return = df_daily_return.reset_index()
    df_daily_return.loc[0, "daily_return"] = 0.0
    df_daily_return = df_daily_return[["date", "daily_return"]]

    # === Step 8: Save Daily Return CSV ===
    df_daily_return.to_csv(output_csv_full_path, index=False)
    print(f"[INFO] MVO daily returns saved to {output_csv_full_path}")

    return df_daily_return


In [183]:
TRAIN_START_DATE = '2015-02-02'
TRAIN_END_DATE = '2023-04-04'
TRADE_START_DATE = '2023-04-05'
TRADE_END_DATE = '2025-04-10'

df_daily_return_mvo = run_mvo_portfolio_pipeline(
    df=processed_0,
    train_start_date='2007-06-01',
    train_end_date=TRAIN_END_DATE,
    trade_start_date=TRADE_START_DATE,
    trade_end_date=TRADE_END_DATE,
    initial_fund=1_000_000,
    weight_bounds=(0.01, 0.25),
    output_return_csv='df_daily_return_mvo.csv',
    original_csv_path='2007-2025_no_crypto.csv'
)

df_daily_return_mvo = run_mvo_portfolio_pipeline(
    df=processed_1,
    train_start_date=TRAIN_START_DATE,
    train_end_date=TRAIN_END_DATE,
    trade_start_date=TRADE_START_DATE,
    trade_end_date=TRADE_END_DATE,
    initial_fund=1_000_000,
    weight_bounds=(0.01, 0.25),
    output_return_csv='df_daily_return_mvo.csv',
    original_csv_path='2015-2025_crypto.csv'
)

df_daily_return_mvo = run_mvo_portfolio_pipeline(
    df=processed_2,
    train_start_date=TRAIN_START_DATE,
    train_end_date=TRAIN_END_DATE,
    trade_start_date=TRADE_START_DATE,
    trade_end_date=TRADE_END_DATE,
    initial_fund=1_000_000,
    weight_bounds=(0.01, 0.25),
    output_return_csv='df_daily_return_mvo.csv',
    original_csv_path='2015-2025_no_crypto.csv'
)



# processed_0 = process_csv_to_features('2007-2025_no_crypto.csv')
# processed_1 = process_csv_to_features('2015-2025_crypto.csv')
# processed_2 = process_csv_to_features('2015-2025_no_crypto.csv')

[INFO] Created folder: 2007-2025_no_crypto/mvo
[INFO] Optimized weights saved to 2007-2025_no_crypto/mvo/optimized_weights.json
[INFO] MVO daily returns saved to 2007-2025_no_crypto/mvo/df_daily_return_mvo.csv
[INFO] Created folder: 2015-2025_crypto/mvo
[INFO] Optimized weights saved to 2015-2025_crypto/mvo/optimized_weights.json
[INFO] MVO daily returns saved to 2015-2025_crypto/mvo/df_daily_return_mvo.csv
[INFO] Created folder: 2015-2025_no_crypto/mvo
[INFO] Optimized weights saved to 2015-2025_no_crypto/mvo/optimized_weights.json
[INFO] MVO daily returns saved to 2015-2025_no_crypto/mvo/df_daily_return_mvo.csv


In [184]:
import os
import pandas as pd

def run_rolling_mvo_pipeline(df,
                              train_start_date,
                              train_end_date,
                              trade_start_date,
                              trade_end_date,
                              window_size=63,
                              train_window_extend=True,
                              initial_fund=1_000_000,
                              weight_bounds=(0.0, 0.5),
                              buy_cost_pct=0.0,
                              sell_cost_pct=0.0,
                              original_csv_path='data.csv',
                              model_name='adaptive_mvo',
                              output_return_csv='df_daily_return_adaptive_mvo.csv'):
    """
    Run Rolling Mean-Variance Optimization (MVO) with rebalancing and export daily returns.
    All outputs are saved in /<csv_name>/<model_name>/.
    """

    # === Step 1: Setup Folder Structure ===
    base_csv_name = os.path.splitext(os.path.basename(original_csv_path))[0]
    target_folder = os.path.join(base_csv_name, model_name)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"[INFO] Created folder: {target_folder}")

    # Adjust output CSV path
    output_return_csv = os.path.join(target_folder, output_return_csv)

    # === Step 2: Data Preparation ===
    train_df = data_split(df, train_start_date, train_end_date).reset_index(drop=True)
    trade_df = data_split(df, trade_start_date, trade_end_date).reset_index(drop=True)

    stock_dimension = len(trade_df.tic.unique())
    unique_dates = trade_df.date.unique()
    total_windows = len(unique_dates) // window_size

    portfolio_values = pd.DataFrame(columns=["account_value"], dtype=float)
    portfolio_dates = []
    weights_log = []

    train_df_window = train_df.copy()

    for w in range(total_windows):
        print(f"\n[Rebalancing] Window {w+1}/{total_windows}")

        start_idx = w * window_size
        end_idx = (w + 1) * window_size
        window_dates = unique_dates[start_idx:end_idx]
        trade_df_window = trade_df[trade_df['date'].isin(window_dates)].copy()

        train_mvo = process_df_for_mvo(train_df_window)
        trade_mvo = process_df_for_mvo(trade_df_window)

        if train_mvo.empty or len(train_mvo) < 2 or train_mvo.shape[1] < 3:
            print(f"[Window {w}] Skipped due to insufficient data.")
            continue

        arReturns = StockReturnsComputing(np.asarray(train_mvo), *train_mvo.shape)
        meanReturns = pd.Series(np.mean(arReturns, axis=0), index=train_mvo.columns)
        covReturns = pd.DataFrame(np.cov(arReturns, rowvar=False), index=train_mvo.columns, columns=train_mvo.columns)

        if meanReturns.std() < 1e-4:
            print(f"[Window {w}] Skipped: flat mean returns.")
            continue

        try:
            ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=weight_bounds)
            ef_mean.max_sharpe()
            cleaned_weights_mean = ef_mean.clean_weights()
        except:
            print(f"[Window {w}] Fallback to min_volatility.")
            ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=weight_bounds)
            ef_mean.min_volatility()
            cleaned_weights_mean = ef_mean.clean_weights()

        weights_log.append(cleaned_weights_mean)

        mvo_weights = np.array([
            initial_fund * (1 - buy_cost_pct) * cleaned_weights_mean[key]
            for key in cleaned_weights_mean.keys()
        ])

        first_prices = trade_mvo.head(1).to_numpy()[0]
        if np.any(first_prices == 0):
            print(f"[Window {w}] Skipped: zero prices detected.")
            continue

        shares = mvo_weights / first_prices
        portfolio_series = trade_mvo @ shares
        MVO_result = pd.DataFrame(portfolio_series, columns=["account_value"])

        dates_in_window = trade_df_window['date'].drop_duplicates().sort_values().tolist()
        portfolio_dates.extend(dates_in_window)
        portfolio_values = pd.concat([portfolio_values, MVO_result], ignore_index=True)

        if train_window_extend:
            train_df_window = pd.concat([train_df_window, trade_df_window], ignore_index=True)

        initial_fund = MVO_result["account_value"].iloc[-1] * (1 - sell_cost_pct)

    # === Step 3: Finalize Portfolio Values ===
    portfolio_values.index = pd.to_datetime(portfolio_dates)

    # Calculate daily returns
    df_daily_return = portfolio_values.copy()
    df_daily_return["daily_return"] = df_daily_return["account_value"].pct_change()
    df_daily_return = df_daily_return.reset_index().rename(columns={"index": "date"})
    df_daily_return.loc[0, "daily_return"] = 0.0
    df_daily_return = df_daily_return[["date", "daily_return"]]

    # Save daily return CSV
    df_daily_return.to_csv(output_return_csv, index=False)
    print(f"[INFO] Rolling MVO daily returns saved to {output_return_csv}")

    # === Optional: Save Weights Log ===
    weights_log_path = os.path.join(target_folder, "weights_log.txt")
    with open(weights_log_path, 'w') as f:
        for idx, weights in enumerate(weights_log):
            f.write(f"Window {idx+1} Weights:\n{weights}\n\n")
    print(f"[INFO] Weights log saved to {weights_log_path}")

    return df_daily_return, weights_log


In [185]:
TRAIN_START_DATE = '2015-02-02'
TRAIN_END_DATE = '2023-04-04'
TRADE_START_DATE = '2023-04-05'
TRADE_END_DATE = '2025-04-10'

df_daily_return, weights_log = run_rolling_mvo_pipeline(
    df=processed_0,
    train_start_date='2007-06-01',
    train_end_date=TRAIN_END_DATE,
    trade_start_date=TRADE_START_DATE,
    trade_end_date=TRADE_END_DATE,
    window_size=63,
    initial_fund=1_000_000,
    weight_bounds=(0.0, 0.5),
    original_csv_path='2007-2025_no_crypto.csv',  
    model_name='adaptive_mvo',
    output_return_csv='df_daily_return_adaptive_mvo.csv'
)

df_daily_return, weights_log = run_rolling_mvo_pipeline(
    df=processed_1,
    train_start_date=TRAIN_START_DATE,
    train_end_date=TRAIN_END_DATE,
    trade_start_date=TRADE_START_DATE,
    trade_end_date=TRADE_END_DATE,
    window_size=63,
    initial_fund=1_000_000,
    weight_bounds=(0.0, 0.5),
    original_csv_path='2015-2025_crypto.csv',   
    model_name='adaptive_mvo',
    output_return_csv='df_daily_return_adaptive_mvo.csv'
)

df_daily_return, weights_log = run_rolling_mvo_pipeline(
    df=processed_2,
    train_start_date=TRAIN_START_DATE,
    train_end_date=TRAIN_END_DATE,
    trade_start_date=TRADE_START_DATE,
    trade_end_date=TRADE_END_DATE,
    window_size=63,
    initial_fund=1_000_000,
    weight_bounds=(0.0, 0.5),
    original_csv_path='2015-2025_no_crypto.csv',   
    model_name='adaptive_mvo',
    output_return_csv='df_daily_return_adaptive_mvo.csv'
)



# processed_0 = process_csv_to_features('2007-2025_no_crypto.csv')
# processed_1 = process_csv_to_features('2015-2025_crypto.csv')
# processed_2 = process_csv_to_features('2015-2025_no_crypto.csv')


[INFO] Created folder: 2007-2025_no_crypto/adaptive_mvo

[Rebalancing] Window 1/8

[Rebalancing] Window 2/8

[Rebalancing] Window 3/8

[Rebalancing] Window 4/8

[Rebalancing] Window 5/8

[Rebalancing] Window 6/8

[Rebalancing] Window 7/8

[Rebalancing] Window 8/8
[INFO] Rolling MVO daily returns saved to 2007-2025_no_crypto/adaptive_mvo/df_daily_return_adaptive_mvo.csv
[INFO] Weights log saved to 2007-2025_no_crypto/adaptive_mvo/weights_log.txt
[INFO] Created folder: 2015-2025_crypto/adaptive_mvo

[Rebalancing] Window 1/8

[Rebalancing] Window 2/8

[Rebalancing] Window 3/8

[Rebalancing] Window 4/8

[Rebalancing] Window 5/8

[Rebalancing] Window 6/8

[Rebalancing] Window 7/8

[Rebalancing] Window 8/8
[INFO] Rolling MVO daily returns saved to 2015-2025_crypto/adaptive_mvo/df_daily_return_adaptive_mvo.csv
[INFO] Weights log saved to 2015-2025_crypto/adaptive_mvo/weights_log.txt
[INFO] Created folder: 2015-2025_no_crypto/adaptive_mvo

[Rebalancing] Window 1/8

[Rebalancing] Window 2/8

[R